In [5]:
#Lord bless our souls
from __future__ import print_function
import tensorflow.keras
from tensorflow.keras import backend
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
import os
import numpy as np
batch_size = 1024
num_classes = 100
epochs = 15

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar100.load_data(label_mode='fine')
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

cimgs = np.concatenate((x_train,x_test))
clbls = np.concatenate((y_train,y_test))
Ncimgs = len(cimgs)
print(Ncimgs)
# Convert class vectors to binary class matrices.
perm = np.random.permutation(np.arange(Ncimgs,dtype=int))
train_x = np.array(list(map(lambda x: cimgs[x], perm[:int(Ncimgs/2)])))
train_y = np.array(list(map(lambda x: clbls[x], perm[:int(Ncimgs/2)])))
test_x = np.array(list(map(lambda x: cimgs[x], perm[int(Ncimgs/2):])))
test_y = np.array(list(map(lambda x: clbls[x], perm[int(Ncimgs/2):])))

train_y = tensorflow.keras.utils.to_categorical(train_y, num_classes)
test_y = tensorflow.keras.utils.to_categorical(test_y, num_classes)

'''
model = Sequential()
model.add(Conv2D(64, (3, 3), padding='same', activation='relu',input_shape=x_train.shape[1:]))
model.add(Conv2D(64, (3, 3),input_shape=(32,32,3),activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512,activation='relu'))
model.add(Dense(256,activation='relu'))
model.add(Dense(num_classes,activation='softmax'))
'''
model = Sequential()
model.add(Conv2D(64, (3, 3), input_shape=(32,32,3), padding='same', activation='relu'))
model.add(Conv2D(64, (3, 3), input_shape=(32,32,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dense(100, activation='softmax'))
# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

x_train shape: (50000, 32, 32, 3)
50000 train samples
10000 test samples
60000


In [6]:
print(model.summary())


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 32, 32, 64)        1792      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 30, 30, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 14400)             0         
_________________________________________________________________
dense_3 (Dense)              (None, 512)               7373312   
_________________________________________________________________
dense_4 (Dense)              (None, 256)               131328    
_________________________________________________________________
dense_5 (Dense)              (None, 100)              

In [7]:
model.fit(train_x, train_y,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(test_x, test_y),
              shuffle=True)

Train on 30000 samples, validate on 30000 samples
Epoch 1/15
30000/30000 [==============================] - 78s 3ms/sample - loss: 19.8812 - acc: 0.0164 - val_loss: 4.5019 - val_acc: 0.0287
Epoch 2/15
30000/30000 [==============================] - 77s 3ms/sample - loss: 4.4086 - acc: 0.0445 - val_loss: 4.3768 - val_acc: 0.0498
Epoch 3/15
30000/30000 [==============================] - 78s 3ms/sample - loss: 4.1951 - acc: 0.0851 - val_loss: 4.2328 - val_acc: 0.0832
Epoch 4/15
30000/30000 [==============================] - 78s 3ms/sample - loss: 3.7677 - acc: 0.1564 - val_loss: 3.9782 - val_acc: 0.1287
Epoch 5/15
30000/30000 [==============================] - 81s 3ms/sample - loss: 3.3067 - acc: 0.2334 - val_loss: 3.9114 - val_acc: 0.1484
Epoch 6/15
30000/30000 [==============================] - 74s 2ms/sample - loss: 2.6744 - acc: 0.3541 - val_loss: 3.9738 - val_acc: 0.1557
Epoch 7/15
30000/30000 [==============================] - 72s 2ms/sample - loss: 2.1441 - acc: 0.4699 - val_loss: 4

In [9]:
model.save("model.hdf5")


array([[3.82014004e-11, 5.54564306e-09, 3.66190491e-08, ...,
        1.14392160e-05, 4.27914802e-06, 1.54943748e-07],
       [3.02720904e-11, 4.16679091e-16, 1.43976892e-15, ...,
        1.22599199e-15, 1.31093955e-15, 4.82813691e-19],
       [9.64035869e-01, 1.48914096e-05, 1.11926667e-04, ...,
        1.08265672e-08, 7.91339616e-09, 5.35670915e-05],
       ...,
       [7.28407406e-13, 1.09313239e-10, 1.81523994e-20, ...,
        8.94238628e-10, 3.01187578e-17, 1.17502026e-08],
       [6.35656625e-06, 2.90984570e-08, 6.93208948e-02, ...,
        4.67709273e-01, 1.98235909e-07, 3.36794801e-05],
       [5.18369745e-07, 1.81172091e-08, 6.15665563e-09, ...,
        1.08250467e-16, 1.24666042e-06, 5.53528013e-12]], dtype=float32)

In [10]:
train_pred = model.predict(train_x)
test_pred = model.predict(test_x)

In [39]:
train_out = np.ones(train_x.shape[0])
test_out = np.zeros(test_x.shape[0])

inputs = np.concatenate((train_pred,test_pred))
outputs = np.concatenate((train_out,test_out))

In [34]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression()
rfe = RFE(logreg, 20)
rfe = rfe.fit(inputs, outputs)
print(rfe.support_)
print(rfe.ranking_)

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

[False False False False  True False False False False False False  True
 False  True  True False False False False False False  True False False
 False False False False False False False  True False False False False
 False False False  True False  True False False False  True False  True
 False False  True False False False  True False  True False False False
 False False False False  True False False False False  True False False
 False False False False False  True  True False False False False False
 False False  True False False  True  True False False False False False
 False False False False]
[63 69 19 36  1 38 21 58 32 74 52  1 24  1  1 14 39 54 12 23 13  1 49  5
 46 10 62  8 78 73  4  1 35 70 47 15 11  9 60  1 25  1  2  7 48  1 33  1
 66 50  1 81 61 30  1 16  1 37 45  3 40 44 71 59  1 20 51 55 22  1  6 67
 27 64 76 57 29  1  1 80 77 31 28 18 53 42  1 41 72  1  1 68 43 34 65 75
 17 79 56 26]


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [40]:
inputs = inputs[:,rfe.support_]
print(inputs.shape)

(60000, 20)


In [42]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

X_train, X_test, y_train, y_test = train_test_split(inputs, outputs, test_size=0.3, random_state=5, shuffle=True)
logreg = LogisticRegression()
logreg.fit(X_train, y_train)


/usr/local/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [43]:
y_pred = logreg.predict(X_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(X_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.48


In [31]:
from sklearn.metrics import classification_report
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.54      0.54      0.54     15060
         1.0       0.54      0.53      0.54     14940

    accuracy                           0.54     30000
   macro avg       0.54      0.54      0.54     30000
weighted avg       0.54      0.54      0.54     30000

